# Scripting languages: practical session 7

22 November 2023

Note: part of this notebook is based on previous tutorials by Jake Vanderplas.

## Pattern Matching with Regular Expressions

The methods of Python's ``str`` object give you a powerful set of tools for formatting, splitting, and manipulating string data.
But even more powerful tools are available in Python's built-in *regular expression* module.

Regular expressions are not exclusive to Python; they are a general means to search for patterns within a sequence of characters. In this practical session, we will explore the types of problems that might be addressed using regular expressions, and we will explore how to create regular expressions and apply them within Python.

We saw how regular expressions are a means of *flexible pattern matching* in strings. In previous sessions, we made use of the `glob` module in order to select filenames. Here, we performed a similar type of flexible matching: we made use of the "``*``" character, which acts as a wildcard.
For example, we listed all text files in a directory by using an expression like `*.txt` in order to select all files with extension *.txt*.

Regular expressions generalize this "wildcard" idea to a wide range of flexible string-matching sytaxes.
The Python interface to regular expressions is contained in the built-in ``re`` module; as a simple example, let's use it to duplicate the functionality of the string ``split()`` method:

In [ ]:
import re

regex = '\s+'
re.split(regex, 'the quick brown fox jumped over a lazy dog')

In [ ]:
help(re)

In the example above, we make use of a regular expression in order to *split* a string.
Just as Python's ``split()`` method returns a list of all substrings between whitespace, the regular expression ``split()`` method returns a list of all substrings between matches to the input pattern.

In this case, the regular expression we use is ``"\s+"``: "``\s``" is a special character that matches any whitespace (space, tab, etc.), and the "``+``" is a character that indicates *one or more* of the entity preceding it.
Thus, the regular expression matches any substring consisting of one or more whitespaces.

The ``split()`` method here is basically a convenience routine built upon this *pattern matching* behavior; more fundamental is the ``match()`` method, which will tell you whether the beginning of a string matches the pattern:

In [ ]:
for s in ["     ", "abc  ", "  abc"]:
    if re.match(regex, s):
        print(f'"{s}" matches')
    else:
        print(f'"{s}" does not match')

The `match()`method will check whether the beginning of the string matches the pattern. If you want to scan an entire string, in order to check whether any substring matches the regular expression pattern, you need to use the `search()` method:

In [ ]:
line = 'the quick brown fox jumped over a lazy dog'

regex = 'fox'
match = re.match(regex, line)
print(f'match method returns: {match}')
search = re.search(regex, line)
print(f'search method returns: {search}')


If you want to substitute a certain substring based on a regular expression, you can make use of the ``sub()`` method:

In [ ]:
re.sub(regex, 'BEAR', line)

### A more sophisticated example

Of course, using regular expressions to look for literal strings (like we did for *fox*) is not very interesting.  We'll now consider a more complicated example: the task of matching email addresses. Let's make use of the (somewhat indecipherable) regular expression below in order to to automatically extract email addressess. If the expression itself is not entirely clear at this point, dont't worry about it; we will explore the different parts of the regular expressions in the course of this notebook.

In [ ]:
email_regex = r'\w+@\w+\.[a-z]{2,3}'

Using this, if we're given a line from a document, we can quickly extract things that look like email addresses

In [ ]:
text = "You can mail Santa Claus using the address santa@thenorthpole.org; alternatively the older address santaclaus@lapland.fi still works as well."
re.findall(email_regex, text)

We can do further operations, like replacing these email addresses with another string, perhaps to hide addresses in the output:

In [ ]:
re.sub(email_regex, 'anonymous@xyz.xyz', text)

Finally, note that if you really want to match *any* email address, the preceding regular expression is far too simple.
For example, it only allows addresses made of alphanumeric characters that end in one of several common domain suffixes.
So, for example, the period used here means that we only find part of the address:

In [ ]:
re.findall(email_regex, 'barack.obama@whitehouse.gov')

This goes to show how unforgiving regular expressions can be if you're not careful!
If you search around online, you can find some suggestions for regular expressions that will match *all* valid emails, but beware: they are much more involved than the simple expression used here!

### Basics of regular expression syntax

#### Simple strings are matched directly

If you build a regular expression on a simple string of characters or digits, it will match that exact string:

In [ ]:
regex = r'ion'
re.findall(regex, 'Great Expectations')

#### Some characters have special meanings

While simple letters or numbers are direct matches, there are a handful of characters that have special meanings within regular expressions. They are:
```
. ^ $ * + ? { } [ ] \ | ( )
```
We will discuss the meaning of some of these momentarily.
In the meantime, you should know that if you'd like to match any of these characters directly, you can *escape* them with a back-slash:

In [ ]:
regex = r'\$'
re.findall(regex, "the cost is $20")

The ``r`` preface in ``r'\$'`` indicates a *raw string*; in standard Python strings, the backslash is used to indicate special characters.
For example, a tab is indicated by ``"\t"``:

In [ ]:
print('a\tb\tc')

Such substitutions are not made in a raw string:

In [ ]:
print(r'a\tb\tc')

For this reason, whenever you use backslashes in a regular expression (and even without them), it is good practice to use a raw string when you define a regular expression.

#### Special characters can match character groups

Just as the ``"\"`` character within regular expressions can escape special characters (turning them into normal characters) it can also be used to give normal characters special meaning.
These special characters match specified groups of characters, and we've seen them before.
In the email address regexp from before, we used the character ``"\w"``, which is a special marker matching *any alphanumeric character*. Similarly, in the simple ``split()`` example, we also saw ``"\s"``, a special marker indicating *any whitespace character*.

Putting these together, we can create a regular expression that will match *any two letters/digits with whitespace between them*:

In [ ]:
regex = r'\w\s\w'
re.findall(regex, 'the fox is 9 years old')

This example begins to hint at the power and flexibility of regular expressions.

The following table lists a few of these characters that are commonly useful:

| Character | Description                 || Character | Description                     |
|-----------|-----------------------------||-----------|---------------------------------|
| ``"\d"``  | Match any digit             || ``"\D"``  | Match any non-digit             |
| ``"\s"``  | Match any whitespace        || ``"\S"``  | Match any non-whitespace        |
| ``"\w"``  | Match any alphanumeric character || ``"\W"``  | Match any non-alphanumeric character |
| ``"\b"``  | Matches a word boundary |

This is *not* a comprehensive list or description; for more details, see Python's [regular expression syntax documentation](https://docs.python.org/3/library/re.html#re-syntax).

#### Square brackets match custom character groups

If the built-in character groups aren't specific enough for you, you can use square brackets to specify any set of characters you're interested in.
For example, the following will match any lower-case vowel:

In [ ]:
regex = r'[aeiou]'
re.findall(regex, 'consequential')

In order to match the inverse, you can use the `^` as the first character within your square brackets.

In [ ]:
regex = r'[^aeiou]'
re.findall(regex, 'consequential')

Similarly, you can use a dash to specify a range: for example, ``"[a-z]"`` will match any lower-case letter, and ``"[1-3]"`` will match any of ``"1"``, ``"2"``, or ``"3"``.
For instance, you may need to extract from a document specific numerical codes that consist of a capital letter followed by a digit. You could do this as follows:

In [ ]:
regex = r'[A-Z][0-9]'
re.findall(regex, '1043879, G2, H6')

#### Wildcards match repeated characters

If you would like to match a string with, say, three alphanumeric characters in a row, it is possible to write, for example, ``"\w\w\w"``.
Because this is such a common need, there is a specific syntax to match repetitions – curly braces with a number:

In [ ]:
regex = r'\w{3}'
re.findall(regex, 'The quick brown fox')

There are also markers available to match any number of repetitions – for example, the ``"+"`` character will match *one or more* repetitions of what precedes it:

In [ ]:
regex = r'\w+'
re.findall(regex, 'The quick brown fox')

The following is a table of the repetition markers available for use in regular expressions:

| Character | Description | Example |
|-----------|-------------|---------|
| ``?`` | Match zero or one repetitions of preceding  | ``"ab?"`` matches ``"a"`` or ``"ab"`` |
| ``*`` | Match zero or more repetitions of preceding | ``"ab*"`` matches ``"a"``, ``"ab"``, ``"abb"``, ``"abbb"``... |
| ``+`` | Match one or more repetitions of preceding  | ``"ab+"`` matches ``"ab"``, ``"abb"``, ``"abbb"``... but not ``"a"`` |
| ``{n}`` | Match ``n`` repetitions of preeeding | ``"ab{2}"`` matches ``"abb"`` |
| ``{m,n}`` | Match between ``m`` and ``n`` repetitions of preceding | ``"ab{2,3}"`` matches ``"abb"`` or ``"abbb"`` |

With these basics in mind, let's return to our email address matcher:

In [ ]:
email_regex = r'\w+@\w+\.[a-z]{2,3}'

We can now understand what this means: we want one or more alphanumeric character (``"\w+"``) followed by the *at sign* (``"@"``), followed by one or more alphanumeric character (``"\w+"``), followed by a period (``"\."`` – note the need for a backslash escape), followed by two or three lower-case letters.

If we want to now modify this so that the Obama email address matches, we can do so using the square-bracket notation:

In [ ]:
email_regex2 = r'[\w.]+@\w+\.[a-z]{2,3}'
re.findall(email_regex2, 'barack.obama@whitehouse.gov')

We have changed ``"\w+"`` to ``"[\w.]+"``, so we will match any alphanumeric character *or* a period.
With this more flexible expression, we can match a wider range of email addresses (though still not all).

#### Parentheses indicate *groups* to extract

For compound regular expressions like our email matcher, we often want to extract their components rather than the full match. This can be done using parentheses to *group* the results:

In [ ]:
email_regex3 = r'([\w.]+)@(\w+)\.([a-z]{2,3})'

In [ ]:
text = "You can mail Santa Claus using the address santa@thenorthpole.org; alternatively the older address santaclaus@lapland.fi still works as well."
re.findall(email_regex3, text)

As we see, this grouping actually extracts a list of the sub-components of the email address, represented as a tuple.

Combining these ideas allows you to flexibly and quickly extract information from strings in Python.

#### A couple of final notes

`^` and `$` respectively indicate the beginning and end of a string. Note that the `^` is different from the `^` used within square brackets, which indicated the inverse of character set.

In [ ]:
text = 'fox1 fox2 fox3'

In [ ]:
re.findall(r'fox[0-9]', text)

In [ ]:
re.findall(r'^fox[0-9]', text)

In [ ]:
re.findall(r'fox[0-9]$', text)

And finally, you can use a backslash with numbers in order to refer to groups you defined earlier in your regular expression.

In [ ]:
text = 'this this string string repeats repeats words words twice'
re.findall(r'(\w+)\s\1', text)

### Further Resources on Regular Expressions

Wednesday's lecture and today's practical session cover the most important elements of regular expressions. If you'd like to learn more, you can find more information in the following resources:

- [Python's ``re`` package Documentation](https://docs.python.org/3/library/re.html): When you start using regular expressions, you will often need to look up what a particular character or sequence means, and how you can properly use it in your regular expression. Once you know the basics, the Python documentation on regular expressions is an incredibly valuable resource to recall what each specific character or sequence means within a regular expression.
- [Python's official regular expression HOWTO](https://docs.python.org/3/howto/regex.html): a more narrative approach to regular expressions in Python.

## Exercises

Now that we've covered the basics on regular expressions, try to apply your knowledge by solving the exercises below. Your job is to fill in the missing regular expression in the exercises. If your regular expression is correct, the conditional expression will print out `Correct`.

### Exercise 1

Write a regular expression that matches all three letter words which start with *h* and end with *t*.

In [ ]:
input_string = 'hot top hit pop cat hut hat'

# add your regular expression between the quotations marks below
data = re.findall(r'h.t', input_string)

print('Result:', data)
if data == ['hot', 'hit', 'hut', 'hat']:
  print('Correct')

### Exercise 2

Write a regular expression that matches all telephone numbers (sequence of numbers starting with a `+`)

In [ ]:
input_string = "My Belgian phone number is +32478203404. And my Dutch one is +316902458433."

# add your regular expression between the quotations marks below
data = re.findall(r'\+\d+',input_string)

print('Result:', data)
if data == ['+32478203404', '+316902458433']:
  print('Correct')

### Exercise 3

Match all words between brackets; make sure that the brackets themselves are not included in the returned result.

In [ ]:
input_string = 'this string (contains) a number of (words) between (brackets)'

# add your regular expression between the quotations marks below
data = re.findall(r'\((\w+)\)', input_string)

print('Result:', data)
if data == ['contains', 'words', 'brackets']:
  print('Correct')

### Exercise 4

From the string defined below, only keep the lines that start with a number or a letter after a `>` sign.

In [ ]:
input_string="""
>Venues
>snowboarding
>[1]
>Netherlands
>[2]
<$#%#$%
<#$#$$
<**&&^^
>Slovenia
>Belgium
>1964
>1968
"""

# add your regular expression between the quotations marks below
data=re.findall('>\w+', input_string)

print('Results:', data)
if data == ['>Venues', '>snowboarding', '>Netherlands', '>Slovenia', '>Belgium', '>1964', '>1968']:
  print('Correct')

### Exercise 5

Extract all words that are exactly 8 characters long.

In [ ]:
input_string="""Au pays parfume que le soleil caresse,
J'ai connu, sous un dais d'arbres tout empourpres
Et de palmiers d'ou pleut sur les yeux la paresse,
Une dame creole aux charmes ignores.

Son teint est pâle et chaud ; la brune enchanteresse
A dans le cou des airs noblement maniérés ;
Grande et svelte en marchant comme une chasseresse,
Son sourire est tranquille et ses yeux assurés."""

# add your regular expression between the quotations marks below
data=re.findall(r':\b\w{8}\b', input_string)

print('Results:', data)
if data == ['palmiers', 'maniérés', 'marchant']:
  print('Correct')

### Exercise 6

The string below contains a number of stock symbols (e.g. TSLA) and the accompanying stock market price. Write a regular expression that extracts both the stock symbol and the accompanying stock market price. Your result should be a list of *(symbol, price)* tuples.

In [ ]:
input_string="""Some of the prices were as following TSLA:749.50, ORCL: 50.50, GE: 10.90,
MSFT:170.50, BIDU: 121.40. As the macroeconomic developments continue we will
update the prices. """

# add your regular expression between the quotations marks below
data=re.findall(r'([A-Z]+):\s?(\d+\.\d+)', input_string)

print('Results:', data)
if data == [('TSLA', '749.50'), ('ORCL', '50.50'), ('GE', '10.90'), ('MSFT', '170.50'), ('BIDU', '121.40')]:
  print('Correct')

### Exercise 7

Write a regular expression that extracts the year, month, and day from the urls below. Eg. for the first url, you should return the tuple `('2021', '10', '03')`.

In [ ]:
input_string="""
https://www.washingtonpost.com/politics/trump-2024-campaign-candidacy/2021/10/03/73af3b12-21f8-11ec-b3d6-8cdebe60d3e2_story.html
https://www.washingtonpost.com/news/football-insider/wp/2016/09/02/odell-beckhams-fame-rests-on-one-stupid-little-ball-josh-norman-tells-author/
https://www.washingtonpost.com/local/pandas-leaving-national-zoo/2020/12/07/a9b0bf68-366e-11eb-8d38-6aea1adb3839_story.html
https://www.washingtonpost.com/lifestyle/food/these-5-bottles-show-that-great-bordeaux-wine-neednt-be-expensive/2018/03/29/4916c11a-32d5-11e8-8abc-22a366b72f2d_story.html
https://www.washingtonpost.com/national/space-shuttle-endeavour-on-schedule-for-monday-launch/2011/05/16/AFa2xl4G_story.html
https://www.washingtonpost.com/world/covid-19-cases-spike-in-belgium-govt-poised-for-action/2021/10/26/42e19822-3639-11ec-9662-399cfa75efee_story.html
"""

# add your regular expression between the quotations marks below
data=re.findall(r'\/(\d{4})/(\d{2})/(\d{2})', input_string)

print('Results:', data)
if data == [('2021', '10', '03'), ('2016', '09', '02'), ('2020', '12', '07'), ('2018', '03', '29'), ('2011', '05', '16'), ('2021', '10', '26')]:
  print('Correct')